In [42]:
import polars as pl
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

dataPath = ""

In [43]:
#Converting columns into appropriate datatypes:
def set_table_dtypes(df: pl.DataFrame) -> pl.DataFrame:
    for col in df.columns:
        # converting the string columns which are actually floating points into floating point columns
        if (col[-1] in ("P", "A")) or (col[:3]=='num' and df[col].dtype==pl.String):
            df = df.with_columns(pl.col(col).cast(pl.Float64).alias(col))
    return df

def convert_strings(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:  
        if df[col].dtype.name in ['object', 'string']:
            df[col] = df[col].astype("string").astype('category')
            current_categories = df[col].cat.categories
            new_categories = current_categories.to_list() + ["Unknown"]
            new_dtype = pd.CategoricalDtype(categories=new_categories, ordered=True)
            df[col] = df[col].astype(new_dtype)
    return df

#Counting how much percent data is null
def null_percent(df: pl.DataFrame) -> pl.DataFrame:
    Ny = df.shape[0]  #Number of rows (or how tall is it)

    nulldf_dict = {}
    for col in df.columns:
        
        x = round((100*df[col].is_null().sum())/Ny)
        
        #create a dictionary for the column names and their null values
        nulldf_dict[col] = [x]
    
    nulldf = pd.DataFrame(data = nulldf_dict)
    return nulldf

def nulldropper(df: pl.DataFrame) -> pl.DataFrame:
    #This function deletes the columns in which null values exceed a threshold (which is 60% at this time)
    Ny = df.shape[0]  #Number of rows (or how tall is it)
    
    #col_after_deletion is list of columns that will survive after dropping nulls
    col_after_deletion = []
    col_delete = []
    #Threshold for null percent
    threshold = 60.0
    
    for col in df.columns:
        
        x = round((100*df[col].is_null().sum())/Ny)
        if x < threshold:
            col_after_deletion.append(col)
        else:
            #metadata containing columns to be deleted.
            col_delete.append(col)
                
    df_new = df.select(col_after_deletion)
    
    return df_new, col_delete
    

def str_to_date(df: pl.DataFrame) -> pl.DataFrame:
    #This function converts string columns which have dates into datetime format
    D_col = []
    for col in df.columns:
        #Note that all the date columns end with a 'D'
        if col[-1]=='D':
            D_col.append(col)
            
    df = df.with_columns([df[col].str.strptime(pl.Datetime, "%Y-%m-%d").alias(col)  for col in D_col])
        
    return df

def extract_string_columns(df: pl.DataFrame) -> pl.DataFrame:
    #This function just outputs the column names and index as a dictionary for those columns which are in string type.
    string_cols = {}
    for i,col in enumerate(df.columns):
        if df[col].dtype==pl.String:
            string_cols[col] = i
    
    return string_cols

def string_columns_list(df: pl.DataFrame) -> pl.DataFrame:
    #This outputs the names of the columns which are in string format.
    string_cols = []
    for i,col in enumerate(df.columns):
        if df[col].dtype==pl.String:
            string_cols.append(col)
    
    return string_cols

def bool_columns_list(df: pl.DataFrame) -> pl.DataFrame:
    #This outputs the names of the columns which are in string format.
    bool_cols = []
    for i,col in enumerate(df.columns):
        if df[col].dtype==pl.Boolean:
            bool_cols.append(col)
    
    return bool_cols

def date_columns_list(df: pl.DataFrame) -> pl.DataFrame:
    #This function outputs the names of the columns which are in datetime format
    date_cols = []
    for i,col in enumerate(df.columns):
        if df[col].dtype==pl.Datetime:
            date_cols.append(col)
    
    return date_cols

def float_columns_list(df: pl.DataFrame) -> pl.DataFrame:
    #This function outputs the names of the columns which are in float64 format.
    float_cols = []
    for i,col in enumerate(df.columns):
        if df[col].dtype==pl.Float64:
            float_cols.append(col)
    
    return float_cols

def numeric_columns_list(df: pl.DataFrame) -> pl.DataFrame:
    
    #This function outputs the names of the columns which are numeric. 
    numeric_cols = []
    
    for i,col in enumerate(df.columns):
        if df[col].dtype.is_numeric():
            numeric_cols.append(col)
    
    return numeric_cols

In [44]:
def compress_unique_values(df: pl.DataFrame) -> pl.DataFrame:
    #We want to find the frequency distribution of the unique string values. 
    df_height = df.shape[0]
    Ny = df_height
    column_dropped = []
    stringindex = []
    
    #This is a dictionary where keys are the column names and values are list of unique string values that are kept after the compression treatment.
    col_unique_values = {}
    for i,col in enumerate(df.columns):
        if df[col].dtype==pl.String:
            
            #x is null percentage
            x = round((100*df[col].is_null().sum())/(df.shape[0]))
            
            #y = number of unique values in column=col
            y = df.unique(subset=col, maintain_order=True).shape[0]
            
            #A is frequency distribution of unique values
            A = df[col].value_counts()
            
            #B sorts A according to count
            B = A.sort('count')
            
            #z is the maximum counts of a unique value
            z = A.select(pl.max('count'))
            
            #mod here is the fraction of max frequent element i.e. the actual mod
            mod = (100*z[0,0])/(Ny)
            
            #Grouping
            frequency_df = df.group_by(col).count().sort(by='count', descending=True)
            
            #Making a dataframe low_freq_values (and then a list low_freq_list) of string values that occur are less than mode/10.
            low_freq_values = frequency_df.filter(pl.col('count') < z/10).select(col)
            low_freq_list = low_freq_values.to_series().to_list()
            
            high_freq_values = frequency_df.filter(pl.col('count') >= z/10).select(col)
            high_freq_list = high_freq_values.to_series().to_list()
            #high_freq_with_none = high_freq_list.append('NONE')
            #Appending the list of unique values that will be left
            col_unique_values[col] = high_freq_list
            
            df = df.with_columns(pl.when(pl.col(col).is_in(low_freq_values)).then(pl.lit('other')).otherwise(pl.col(col)).alias(col))
            
            #making a list of columns with string datatype
            stringindex.append(i)
            
            if mod>90:
                #if the mod covers 90% or more of a column then it is useless as a column
                column_dropped.append(col)
            
    df = df.drop(column_dropped)
    return df, column_dropped, col_unique_values

In [45]:
#Zero variance remover is needed.
#If the variance of a column is zero, then it must be removed.
def zero_variance_remover(df: pl.DataFrame) -> pl.DataFrame:
    
    #Getting the list of columns with float type data
    numeric_cols = numeric_columns_list(df)
    
    #Creating a dataframe A with numeric columns
    A = df[numeric_cols]
    
    #Creating a dataframe V (with just one row) which has the standard dev of each column of A
    V = A.std()
    
    column_dropped = []
    for col in numeric_cols:
        if V[col][0]==0:
            column_dropped.append(col)

    df = df.drop(column_dropped)
    return df, column_dropped

In [46]:
#Remove columns with high correlations,
def high_corr_remover(df: pl.DataFrame) -> pl.DataFrame:
    
    #Getting the list of columns with float type data
    float_cols = float_columns_list(df)
    
    #Creating a dataframe A with only float type columns of df
    A = df[float_cols]
    
    #Initializing a list of columns which will be dropped
    column_dropped = []
    
    #Two nested loops for finding correlation between i^th and j^th column. If the correlation between two columns is greater than 0.9 (90%)
    #then we delete the column with greater number of null values among the two.
    for i,col1 in enumerate(A.columns):
        for j,col2 in enumerate(A.columns):
            if j>=i:
                break
            else:
                B=A.select(pl.corr(col1, col2))
                #Let's set 0.9 as the threshold.
                if B[0,0]>0.9:
                    #If null value of col2 is greater then append col 2 in the list of columns to be deleted else do otherwise.
                    if (A[col1].is_null().sum())<(A[col2].is_null().sum()):
                        column_dropped.append(col2)
                    else:
                        column_dropped.append(col1)
    
    df = df.drop(column_dropped)
    return df, column_dropped

In [47]:
def Zscore_normalizer(df: pl.DataFrame) -> pl.DataFrame:
    #This function uses Zscore normalization to normalize all the float type columns.
    
    #Getting the list of columns with float type data
    float_cols = float_columns_list(df)
    
    #Creating a dataframe A with only float type columns of df
    A = df[float_cols]
    
    #Creating a dataframe B whose columns are that of df but Zscore normalized. So each float type column in B now has a "mean = 0" and a "std = 1".
    for col in A.columns:
        if df[col].std()!=0:
            df = df.with_columns((pl.col(col)-pl.col(col).mean())/(pl.col(col).std()))
    
    return df

In [48]:
def subtract_reference_date(df: pl.DataFrame) -> pl.DataFrame:
    
    #Extract the list of columns with date datatype.
    date_col = date_columns_list(df)
    
    #Creating a reference column whose dates are minimum among all corresponding elements of all the other date columns.
    df = df.with_columns(pl.min_horizontal(date_col).alias("min_date"))
    
    #Subtracting the reference date from every date column.
    for col in date_col:
        df = df.with_columns((pl.col(col) - pl.col("min_date")).dt.total_days().alias(col))
    
    #Drop the reference date columns.
    df = df.drop('min_date')
    
    return df

In [49]:
def one_hot_encode(train_df: pl.DataFrame, test_df: pl.DataFrame) -> tuple[pl.DataFrame,pl.DataFrame]:
    encoded_train = train_df
    encoded_test = test_df
    string_cols = train_df.pipe(string_columns_list)
    boolean_cols = train_df.pipe(bool_columns_list)
    catcols = string_cols + boolean_cols
    for colm in catcols:
        if colm != 'case_id':
            # Get unique categories from the training DataFrame
            unique_categories = train_df[colm].unique().to_list()

            # Create one-hot encoding for the training DataFrame
            encoded_train = encoded_train.with_columns(
                [
                    pl.when(pl.col(colm) == category).then(1).otherwise(0).alias(f"{colm}_{category}") if category is not None
                    else pl.when(pl.col(colm).is_null()).then(1).otherwise(0).alias(f"{colm}_None")
                    for category in unique_categories
                ]
            )

            # Drop the original column from the training DataFrame
            encoded_train = encoded_train.drop(colm)

            # Create one-hot encoding for the testing DataFrame
            encoded_test = encoded_test.with_columns(
                [
                    pl.when(pl.col(colm) == category).then(1).otherwise(0).alias(f"{colm}_{category}") if category is not None
                    else pl.when(pl.col(colm).is_null()).then(1).otherwise(0).alias(f"{colm}_None")
                    for category in unique_categories
                ]
            )

            # Drop the original column from the testing DataFrame
            encoded_test = encoded_test.drop(colm)

            # Add missing columns to the test DataFrame
            for category in unique_categories:
                category_col = f"{colm}_{category}" if category is not None else f"{colm}_None"
                if category_col not in encoded_test.columns:
                    encoded_test = encoded_test.with_columns(pl.lit(0).alias(category_col))

            # Ensure the columns are in the same order
            encoded_test = encoded_test.select(encoded_train.columns)

    # Verify the columns
    #print(encoded_train.columns)
    #print(encoded_test.columns)
    return encoded_train,encoded_test

In [50]:
def round_floats(df: pl.DataFrame) -> pl.DataFrame:
    
    #This function will round off all the float type columns in order to make values like 10^-9 into 0.
    float_cols = float_columns_list(df)
    
    for col in float_cols:
        df = df.with_columns(pl.col(col).round(6).alias(col))  
    
    return df

In [51]:
def rename_categories(df: pl.DataFrame, categories_dict: dict) -> pl.DataFrame:
    for col, valid_categories in categories_dict.items():
        if col in df.columns:
            # Create a dictionary for fast lookup
            category_set = set(valid_categories)
            df = df.with_columns(
                pl.col(col).map_elements(lambda x: x if x in category_set else 'other', return_dtype=pl.Utf8).alias(col)
            )
    return df

In [52]:
def combined_pipes_both(train_df,test_df):
    train_df = train_df.pipe(set_table_dtypes).pipe(str_to_date).pipe(subtract_reference_date)
    test_df = test_df.pipe(set_table_dtypes).pipe(str_to_date).pipe(subtract_reference_date)
    
    train_df, columns_dropped = train_df.pipe(nulldropper)
    test_df = test_df.drop(columns_dropped)
    
    #df_train, col_drop, unique_string_values = df_train.pipe(compress_unique_values)
    #df_test = df_test.drop(col_drop)
    
    #train_df = df_train
    #test_df = df_test
    
    # Compressing the categories in each column
    train_df, str_columns_dropped, str_col_unique_values = train_df.pipe(compress_unique_values)
    
    # Replicating the compression of categories in the test data
    test_columns = test_df.columns
    str_columns_pre_1hot = [col for col in test_columns if col not in str_columns_dropped]
    test_df = test_df[str_columns_pre_1hot]
    test_df = test_df.pipe(rename_categories,str_col_unique_values)
    
    # One Hot Encoding
    train_df,test_df = train_df.pipe(one_hot_encode,test_df)
    
    train_df, col_drop = train_df.pipe(zero_variance_remover)
    test_df = test_df.drop(col_drop)
    
    train_df = train_df.pipe(Zscore_normalizer)
    test_df = test_df.pipe(Zscore_normalizer)
    
    train_df, col_drop = train_df.pipe(high_corr_remover)
    test_df = test_df.drop(col_drop)
    
    train_df = train_df.fill_null(strategy="mean")
    means = train_df.mean()
    for col in test_df.columns:
        test_df = test_df.with_columns(pl.when(pl.col(col).is_null()).then(means[col]).otherwise(pl.col(col)).alias(col))
    
    train_df = train_df.pipe(round_floats)
    test_df = test_df.pipe(round_floats)


    return train_df, test_df


In [53]:
dataPath = ""
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(set_table_dtypes)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(set_table_dtypes) 
train_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

In [54]:
ts_cb = train_static.join(train_static_cb, on="case_id", how="outer")

In [55]:
ts_cb

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_629L,applicationscnt_867L,avgdbddpdlast24m_3658932P,avgdbddpdlast3m_4187120P,avgdbdtollast24m_4525197P,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avglnamtstart24m_4525187A,avgmaxdpdlast9m_3716943P,avgoutstandbalancel6m_4187114A,avgpmtlast12m_4525200A,bankacctype_710L,cardtype_51L,clientscnt12m_3712952L,clientscnt3m_3712950L,clientscnt6m_3712949L,clientscnt_100L,clientscnt_1022L,clientscnt_1071L,clientscnt_1130L,clientscnt_136L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,…,firstquarter_103L,for3years_128L,for3years_504L,for3years_584L,formonth_118L,formonth_206L,formonth_535L,forquarter_1017L,forquarter_462L,forquarter_634L,fortoday_1092L,forweek_1077L,forweek_528L,forweek_601L,foryear_618L,foryear_818L,foryear_850L,fourthquarter_440L,maritalst_385M,maritalst_893M,numberofqueries_373L,pmtaverage_3A,pmtaverage_4527227A,pmtaverage_4955615A,pmtcount_4527229L,pmtcount_4955617L,pmtcount_693L,pmtscount_423L,pmtssum_45A,requesttype_4525192L,responsedate_1012D,responsedate_4527233D,responsedate_4917613D,riskassesment_302T,riskassesment_940T,secondquarter_766L,thirdquarter_1082L
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,f64,f64,str,str,f64,f64,f64,str,str,str,str,str,f64,f64,f64
0,null,null,1917.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1,null,null,3134.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0.0""",3.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
2,null,null,4937.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3,null,null,4643.6,0.0,0.0,1.0,0.0,2.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,1.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
4,null,null,3390.2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,null,null,null,null,null,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2703450,0.0,176561.36,3675.4001,0.0,0.0,0.0,0.0,0.0,0.0,10.0,-23.0,-43.0,-23.0,0.0,7356.8003,null,0.0,16392.496,6750.2,"""CA""",null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1.0,"""a55475b1""","""a55475b1""",0.0,null,null,12155.0,null,"""12.0""",null,null,null,null,null,null,"""2020-10-19""",null,null,1.0,1.0
2703451,0.0,301276

In [56]:
ts = ts_cb

In [57]:
train_df, test_df = train_test_split(ts, train_size=0.6, random_state=1)

In [58]:
df1, df2 = train_df.pipe(combined_pipes_both,test_df);

C:\Users\yaman\AppData\Local\Temp\ipykernel_2100\1573512404.py:32: DeprecationWarning: `count` is deprecated. It has been renamed to `len`.
  frequency_df = df.group_by(col).count().sort(by='count', descending=True)


In [59]:
df1

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_867L,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avgmaxdpdlast9m_3716943P,clientscnt12m_3712952L,clientscnt_100L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,credamount_770A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,downpmt_116A,eir_270L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,…,lastrejectreason_759M_P94_109_143,lastrejectreason_759M_a55475b1,lastrejectreasonclient_4145040M_a55475b1,lastrejectreasonclient_4145040M_P94_109_143,lastrejectreasonclient_4145040M_other,lastst_736L_D,lastst_736L_other,lastst_736L_T,lastst_736L_None,lastst_736L_A,lastst_736L_K,previouscontdistrict_112M_P197_47_166,previouscontdistrict_112M_a55475b1,previouscontdistrict_112M_P131_33_167,previouscontdistrict_112M_other,description_5085714M_a55475b1,description_5085714M_None,description_5085714M_2fc785b2,education_1103M_6b2ae0fa,education_1103M_717ddd49,education_1103M_None,education_1103M_a55475b1,education_1103M_other,maritalst_385M_other,maritalst_385M_3439d993,maritalst_385M_a55475b1,maritalst_385M_None,maritalst_385M_a7fcb6e5,requesttype_4525192L_other,requesttype_4525192L_PENSION_6,requesttype_4525192L_None,requesttype_4525192L_DEDUCTION_6,isbidproduct_1095L_False,isbidproduct_1095L_True,opencred_647L_False,opencred_647L_None,opencred_647L_True
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,f64,f64,f64,i64,i64,f64,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
1634941,-0.00663,-0.357659,-0.639665,-0.511118,-0.00159,-0.284331,-0.141311,-0.110718,-0.203828,-0.143455,-0.599751,-0.154994,-0.049361,-0.105379,-0.178587,-0.122337,-0.09899,-0.051145,-0.068424,-0.050928,-0.016045,-0.268305,-0.060627,-0.106447,-0.193459,1.314976,-0.873039,-0.299082,13029,14990,-0.168819,-0.134165,0.62535,14701,11816,0.429246,…,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,0,0
250989,-0.00663,-0.567268,0.310082,0.854127,-0.00159,-0.284331,-0.141311,-0.110718,-0.203828,-0.143455,-0.237808,-0.154994,-0.049361,-0.105379,-0.178587,-0.122337,-0.09899,-0.051145,-0.068424,-0.050928,-0.016045,-0.268305,-0.060627,-0.106447,-0.193459,-0.714512,0.681374,1.677695,14979,15923,-0.2001,-0.134165,0.875876,16424,24983,-0.748928,…,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0
1493791,-0.00663,-0.78141,0.143181,-0.511118,-0.00159,-0.284331,-0.141311,-0.110718,0.257655,-0.143455,-0.0,0.0,-0.049361,-0.105379,-0.178587,-0.122337,-0.09899,-0.051145,-0.068424,-0.050928,-0.016045,-0.268305,-0.060627,-0.106447,-1.002024,-1.348726,0.084909,-0.299082,9043,11369,-0.179246,-0.134165,0.789837,11917,10550,-0.748928,…,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0
962173,0.0,-0.0,0.155515,-0.511118,-0.00159,-0.284331,-0.141311,-0.110718,-0.43457,0.0,-0.0,0.0,-0.049361,-0.105379,-0.178587,-0.122337,1.145629,-0.051145,-0.068424,-0.050928,-0.016045,3.278097,-0.060627,-0.106447,-0.0,0.0,0.572685,-0.299082,14979,15923,-0.0,-0.134165,-0.0,16424,14805,-0.748928,…,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,1,0,0
1531261,-0.00663,-0.78141,-0.220922,-0.511118,-0.00159,-0.284331,-0.141311,-0.110718,-0.665312,1.689649,-0.0,0.0,-0.049361,-0.105379,-0.178587,-0.122337,-0.09899,-0.051145,-0.068424,-0.050928,-0.016045,-0.268305,-0.060627,-0.106447,-1.002024,-1.348726,-0.696783,-0.299082,10528,10926,1.238806,-0.134165,-1.401631,16424,10776,-0.748928,…,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0

In [60]:
df2

case_id,actualdpdtolerance_344P,amtinstpaidbefduel24m_4187115A,annuity_780A,annuitynextmonth_57A,applicationcnt_361L,applications30d_658L,applicationscnt_1086L,applicationscnt_464L,applicationscnt_867L,avgdpdtolclosure24_3658938P,avginstallast24m_3658937A,avgmaxdpdlast9m_3716943P,clientscnt12m_3712952L,clientscnt_100L,clientscnt_1071L,clientscnt_1130L,clientscnt_157L,clientscnt_257L,clientscnt_304L,clientscnt_360L,clientscnt_493L,clientscnt_533L,clientscnt_887L,clientscnt_946L,cntincpaycont9m_3716944L,cntpmts24_3658933L,credamount_770A,currdebtcredtyperange_828A,datefirstoffer_1144D,datelastunpaid_3546854D,daysoverduetolerancedd_3976961L,downpmt_116A,eir_270L,firstclxcampaign_1125D,firstdatedue_489D,homephncnt_628L,…,lastrejectreason_759M_P94_109_143,lastrejectreason_759M_a55475b1,lastrejectreasonclient_4145040M_a55475b1,lastrejectreasonclient_4145040M_P94_109_143,lastrejectreasonclient_4145040M_other,lastst_736L_D,lastst_736L_other,lastst_736L_T,lastst_736L_None,lastst_736L_A,lastst_736L_K,previouscontdistrict_112M_P197_47_166,previouscontdistrict_112M_a55475b1,previouscontdistrict_112M_P131_33_167,previouscontdistrict_112M_other,description_5085714M_a55475b1,description_5085714M_None,description_5085714M_2fc785b2,education_1103M_6b2ae0fa,education_1103M_717ddd49,education_1103M_None,education_1103M_a55475b1,education_1103M_other,maritalst_385M_other,maritalst_385M_3439d993,maritalst_385M_a55475b1,maritalst_385M_None,maritalst_385M_a7fcb6e5,requesttype_4525192L_other,requesttype_4525192L_PENSION_6,requesttype_4525192L_None,requesttype_4525192L_DEDUCTION_6,isbidproduct_1095L_False,isbidproduct_1095L_True,opencred_647L_False,opencred_647L_None,opencred_647L_True
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1.819929e6,-0.007769,-0.153023,-0.947166,0.082766,-0.00378,1.753668,-0.12858,-0.110188,0.486593,-0.141226,-0.139091,0.054296,-0.044309,-0.132659,-0.176176,-0.127745,-0.108167,-0.046637,-0.053731,-0.049672,-0.010019,-0.266548,-0.059825,-0.109317,-0.352623,-0.206065,-0.675158,-0.29861,15596.0,15571.0,-0.194163,-0.135527,-0.0,15666.0,14875.0,-0.749029,…,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
698266.0,0.0,-0.0,-0.346069,-0.512061,-0.00378,-0.280322,-0.12858,-0.110188,-0.658794,0.0,-0.0,0.0,-0.044309,-0.132659,-0.176176,-0.127745,-0.108167,-0.046637,-0.053731,-0.049672,-0.010019,-0.266548,-0.059825,-0.109317,-0.0,0.0,0.003227,-0.29861,14979.121456,15923.181599,-0.0,2.327825,-0.0,16424.281045,14805.119086,-0.749029,…,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
2.634185e6,-0.007769,1.048701,0.657073,-0.512061,-0.00378,5.821647,-0.12858,-0.110188,2.777367,-0.144343,0.493733,-0.156573,-0.044309,-0.132659,-0.176176,-0.127745,-0.108167,-0.046637,-0.053731,-0.049672,-0.010019,3.275727,0.071444,-0.109317,0.293772,0.807133,1.135363,-0.29861,10488.0,12094.0,-0.198296,-0.135527,-0.0,11353.0,10274.0,0.428948,…,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
116564.0,-0.007769,-0.0949,0.733539,1.240371,-0.00378,-0.280322,0.473185,-0.110188,-0.429717,-0.144343,-0.075123,-0.156573,-0.044309,-0.132659,-0.176176,-0.127745,-0.108167,-0.046637,-0.053731,-0.049672,-0.010019,-0.266548,-0.033571,-0.109317,0.61697,-0.079415,0.682744,-0.29861,8277.0,15923.181599,-0.200362,-0.135527,0.874235,8857.0,8119.0,0.428948,…,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1.852036e6,-0.007769,1.044794,0.952241,

In [61]:
test_basetable = pl.read_csv(dataPath + "csv_files/test/test_base.csv")
test_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/test/test_static_0_0.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_1.csv").pipe(set_table_dtypes),
        pl.read_csv(dataPath + "csv_files/test/test_static_0_2.csv").pipe(set_table_dtypes),
    ],
    how="vertical_relaxed",
)
test_static_cb = pl.read_csv(dataPath + "csv_files/test/test_static_cb_0.csv").pipe(set_table_dtypes)
test_person_1 = pl.read_csv(dataPath + "csv_files/test/test_person_1.csv").pipe(set_table_dtypes) 
test_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/test/test_credit_bureau_b_2.csv").pipe(set_table_dtypes) 

## Feature engineering

In this part, we can see a simple example of joining tables via `case_id`. Here the loading and joining is done with polars library. Polars library is blazingly fast and has much smaller memory footprint than pandas. 

In [63]:
# We need to use aggregation functions in tables with depth > 1, so tables that contain num_group1 column or 
# also num_group2 column.
train_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

# Here num_group1=0 has special meaning, it is the person who applied for the loan.
train_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

# Here we have num_goup1 and num_group2, so we need to aggregate again.
train_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

# We will process in this examples only A-type and M-type columns, so we need to select them.
selected_static_cols = []
for col in train_static.columns:
    if col[-1] in ("A", "M"):
        selected_static_cols.append(col)
print(selected_static_cols)

selected_static_cb_cols = []
for col in train_static_cb.columns:
    if col[-1] in ("A", "M"):
        selected_static_cb_cols.append(col)
print(selected_static_cb_cols)

# Join all tables together.
data = train_basetable.join(
    train_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    train_person_1_feats_1, how="left", on="case_id"
).join(
    train_person_1_feats_2, how="left", on="case_id"
).join(
    train_credit_bureau_b_2_feats, how="left", on="case_id"
)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [64]:
test_person_1_feats_1 = train_person_1.group_by("case_id").agg(
    pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"),
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")
)

test_person_1_feats_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    pl.col("num_group1") == 0
).drop("num_group1").rename({"housetype_905L": "person_housetype"})

test_credit_bureau_b_2_feats = train_credit_bureau_b_2.group_by("case_id").agg(
    pl.col("pmts_pmtsoverdue_635A").max().alias("pmts_pmtsoverdue_635A_max"),
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("pmts_dpdvalue_108P_over31")
)

data_submission = test_basetable.join(
    test_static.select(["case_id"]+selected_static_cols), how="left", on="case_id"
).join(
    test_static_cb.select(["case_id"]+selected_static_cb_cols), how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
)

In [65]:
case_ids = data["case_id"].unique().shuffle(seed=1)
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

cols_pred = []
for col in data.columns:
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

print(cols_pred)

def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

for df in [X_train, X_valid, X_test]:
    df = convert_strings(df)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [66]:
print(f"Train: {X_train.shape}")
print(f"Valid: {X_valid.shape}")
print(f"Test: {X_test.shape}")

Train: (915995, 48)
Valid: (305332, 48)
Test: (305332, 48)


## Training LightGBM

In [68]:
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_valid = lgb.Dataset(X_valid, label=y_valid, reference=lgb_train)

params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "max_depth": 3,
    "num_leaves": 31,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "bagging_fraction": 0.8,
    "bagging_freq": 5,
    "n_estimators": 1000,
    "verbose": -1,
}

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_valid,
    callbacks=[lgb.log_evaluation(50), lgb.early_stopping(1000)]
)

C:\Users\yaman\anaconda3\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Training until validation scores don't improve for 1000 rounds
[50]	valid_0's auc: 0.705963
[100]	valid_0's auc: 0.724362
[150]	valid_0's auc: 0.731423
[200]	valid_0's auc: 0.735874
[250]	valid_0's auc: 0.739009
[300]	valid_0's auc: 0.740965
[350]	valid_0's auc: 0.742924
[400]	valid_0's auc: 0.744582
[450]	valid_0's auc: 0.745977
[500]	valid_0's auc: 0.747033
[550]	valid_0's auc: 0.747877
[600]	valid_0's auc: 0.749039
[650]	valid_0's auc: 0.750087
[700]	valid_0's auc: 0.750863
[750]	valid_0's auc: 0.751207
[800]	valid_0's auc: 0.751792
[850]	valid_0's auc: 0.752608
[900]	valid_0's auc: 0.753146
[950]	valid_0's auc: 0.753504
[1000]	valid_0's auc: 0.753937
Did not meet early stopping. Best iteration is:
[984]	valid_0's auc: 0.753951


Evaluation with AUC and then comparison with the stability metric is shown below.

In [70]:
for base, X in [(base_train, X_train), (base_valid, X_valid), (base_test, X_test)]:
    y_pred = gbm.predict(X, num_iteration=gbm.best_iteration)
    base["score"] = y_pred

print(f'The AUC score on the train set is: {roc_auc_score(base_test["target"], base_test["score"])}') 
print(f'The AUC score on the valid set is: {roc_auc_score(base_valid["target"], base_valid["score"])}') 
print(f'The AUC score on the test set is: {roc_auc_score(base_test["target"], base_test["score"])}')  

The AUC score on the train set is: 0.7509685352379327
The AUC score on the valid set is: 0.7539505564652744
The AUC score on the test set is: 0.7509685352379327
